Как я понял, в данном тестовом задании нужно удобно обработать датасет, чтобы его потом можно было использовать в сочетании llm, после чего придумать промпт, который выводит из данных ответ.

In [ ]:
import pandas as pd
dfile = pd.read_csv("/content/test_dialogs_dataset - test_dialogs_dataset.csv")

In [ ]:
def prepare(sample: str):
  samples = sample.replace("[", "\n[").split('\n') #разделение по квадратной скобке
  samples = [sample[21:].strip() for sample in samples] #убираем даты
  samples = [sample for sample in samples if sample[:6] == "Клиент" or sample[:8] == "Менеджер"] #проверяем какое начало у строк
  dialog = []
  for sample in samples:
    dialog.append((sample.split(":")[0].strip(), sample.split(":")[1].strip()))
  return dialog

ds = [dfile["dialog_text"][i] for i in range(len(dfile))]
ds = list(map(prepare, ds))

[('Менеджер', 'Добрый день!!'), ('Клиент', 'Здравствуйте! Хотел бы узнать о новой акции... 😊🙂'), ('Менеджер', 'Конечно, у нас есть отличное предложение!!!!---')]


теперь датасет выглядит так

In [ ]:
print(ds)

[[('Менеджер', 'Добрый день!!'), ('Клиент', 'Здравствуйте! Хотел бы узнать о новой акции... 😊🙂'), ('Менеджер', 'Конечно, у нас есть отличное предложение!!!!---')], [('Менеджер', 'Здравствуйте, чем могу помочь?'), ('Клиент', 'я хотел бы отменить свой заказ.... ***'), ('Менеджер', 'Хорошо, я помогу вам с этим!!!! 123')], [('Клиент', 'Я ищу информацию о вашем продукте... ?!!'), ('Менеджер', 'Конечно! Вот ссылка на наш сайт')]]


у меня нету доступа к openai, поэтому в целях демонстрации я использую модель ruGPT3Large.

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
tok, model = GPT2Tokenizer.from_pretrained("sberbank-ai/rugpt3large_based_on_gpt2"), GPT2LMHeadModel.from_pretrained("sberbank-ai/rugpt3large_based_on_gpt2").cuda()

In [ ]:
def generate(
    model, tok, text,
    do_sample=True, max_length=100, repetition_penalty=5.0,
    top_k=5, top_p=0.95, temperature=1,
    num_beams=None,
    no_repeat_ngram_size=3
    ):
  input_ids = tok.encode(text, return_tensors="pt").cuda()
  out = model.generate(
      input_ids.cuda(),
      max_length=max_length,
      repetition_penalty=repetition_penalty,
      do_sample=do_sample,
      top_k=top_k, top_p=top_p, temperature=temperature,
      num_beams=num_beams, no_repeat_ngram_size=no_repeat_ngram_size
      )
  return list(map(tok.decode, out))

Использование нового промпта

In [ ]:
old_prompt = "придумай задание для менеджера на основе этого диалога"
new_prompt = "придумай задание, которое может прийти в crm, для менеджера на основе этого диалога"

In [ ]:
for dialog in ds:
  prompt = ''
  prompt += new_prompt
  prompt += '\n'
  for x,y in dialog:
    prompt += x
    prompt += ":"
    prompt += y
    prompt += "\n"
  generated = generate(model=model, tok=tok, text=prompt, num_beams=10)[0]
  print(generated)
  print()

придумай задание, которое может прийти в crm, для менеджера на основе этого диалога
Менеджер:Добрый день!!
Клиент:Здравствуйте! Хотел бы узнать о новой акции... 😊🙂
Менеджер:Конечно, у нас есть отличное предложение!!!!---
Договоритесь с менеджером по телефону и получите скидку 10% при заказе от 50 000 руб.<s>
Узнай как замшелые убеждения, стереотипы

придумай задание, которое может прийти в crm, для менеджера на основе этого диалога
Менеджер:Здравствуйте, чем могу помочь?
Клиент:я хотел бы отменить свой заказ.... ***
Менеджер:Хорошо, я помогу вам с этим!!!! 123
Договариваемся о том, что менеджер перезвонит клиенту и сообщит об отмене заказа. Менеджер перезванивает клиенту и сообщает об отмене его заказа. Клиент звонит менеджеру и отменяет

придумай задание, которое может прийти в crm, для менеджера на основе этого диалога
Клиент:Я ищу информацию о вашем продукте...?!!
Менеджер:Конечно! Вот ссылка на наш сайт
клиент:Спасибо. А что вы можете мне предложить?
менеджер:Давайте я вам расскажу

Для того, чтобы узнать хорошо или плохо сработал промпт, мы должны иметь датасет эталонных ответом и сравнивать выход от двух промптов с эталонным. Для сравнения похожести на эталонный ответ можно использовать мультиязычную версию BERT Score.

ЗАДАНИЕ 2

Чтобы реализовать самостоятельное закрывание некоторых задач сервисом, нам следует понять в каком случае необходимо совершить то или иное действие. Например, нужно научить сервис в нужный момент напоминать клиенту о встрече. Для этого можно создать датасет диалогов и каждому в соответствие поставить флаг, нужно ли в этом случае делать напоминание. После можно обучить модель на этом датасете и встроить ее в сервис. Так как мы не можем доверить принятие решения нейросети, это можно оформить в форме подсказок от ИИ. Либо можно выделить отдельное пространство, в которое будут приходить нейросетевые рекомендации.

Допустим, надо сделать так, чтобы сервис напоминал о том, что клиент тебе долгое время не отвечает. Для этого можно таким же образом проанализировать множество ситуаций, когда уместно или неуместно напомнить о себе. Необходимо использовать информацию с датой и временем отправки сообщений.

Нужно чтобы сервис постоянно оценивал состояние твоих диалогов, чтобы понять, нужно ли в данный момент дать некоторую подсказку или нет. Как это сделать? Может быть, нужно, чтобы сервис в каждом дилоге отправлял в модель последние n сообщений, после чего модель выдаст, какое действие лучше совершить в этом конкретом диалоге.